In [ ]:
# import matplotlib
# matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import numpy as np
import umap
import torch

from sklearn.manifold import TSNE

In [ ]:
from src.utils.utils_model import load_model_specific_params
from src.utils.prepare import prepare_model

def load_model(model_name, checkpoint_path):
    model_params = load_model_specific_params(model_name)
    model_params = {
        'num_classes': 10,
        'input_channels': 3,
        'img_height': 32,
        'img_width': 32,
        'overlap': 00,
        **model_params
        }
        
    model = prepare_model(model_name, model_params=model_params)
    model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
    return model


def trace_of_total_covariance(neurons):
    mean_neuron = neurons.mean(axis=0)
    neurons -= mean_neuron
    cov_matrix = (neurons.unsqueeze(2) @ neurons.unsqueeze(1)).mean(axis=0)
    trace = torch.trace(cov_matrix)
    return trace

In [ ]:
def display_neurons_2d(model, reducer):

    # 1. Załaduj przetrenowany model ResNet50
    model.eval()

    # Przejrzyj warstwy konwolucyjne i wydobądź ich wagi
    for (name1, layer1), (name2, layer2) in zip(model.left_branch.named_modules(), model.right_branch.named_modules()):
        if isinstance(layer1, torch.nn.Conv2d) and isinstance(layer2, torch.nn.Conv2d):
            # Pobierz wagi
            weights1 = layer1.weight.data.cpu().numpy()
            weights2 = layer2.weight.data.cpu().numpy()

            # Spłaszcz wagi
            num_filters1 = weights1.shape[0]
            num_filters2 = weights2.shape[0]
            flattened_weights1 = weights1.reshape(num_filters1, -1)
            flattened_weights2 = weights2.reshape(num_filters2, -1)
            colors = np.where(np.arange(2 * flattened_weights1.shape[0]) < flattened_weights1.shape[0], 'blue', 'red')

            # 2. Zastosuj UMAP do redukcji wymiarów
            if reducer == 'umap':
                embedding = umap.UMAP(n_components=2, n_neighbors=10, densmap=True).fit_transform(np.concatenate([flattened_weights1, flattened_weights2]))
            elif reducer == 'tsne':
                embedding = TSNE(n_components=2).fit_transform(np.concatenate([flattened_weights1, flattened_weights2]))
            print('DIMS', embedding.shape, flattened_weights1.shape, flattened_weights2.shape)
            
            
            pca1 = PCA()
            singular_values1 = pca1.fit(flattened_weights1).singular_values_
            square_stable_rank1 = round((singular_values1 ** 2).sum() / singular_values1[0] ** 2, 4)
            
            pca2 = PCA()
            singular_values2 = pca2.fit(flattened_weights2).singular_values_
            square_stable_rank2 = round((singular_values2 ** 2).sum() / singular_values2[0] ** 2, 4)

            # 3. Wizualizacja
            # fig = plt.figure()
            # ax = fig.add_subplot(111, projection='3d')
            # ax.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], c=colors, marker='o')
            # plt.title(f'UMAP Visualization of Weights in Layer {name1} which has {num_filters1} neurons and Layer {name2} which has {num_filters2}')
            # ax.set_xlabel('Component 1')
            # ax.set_ylabel('Component 2')
            # ax.set_zlabel('Component 3')
            # plt.show()
            
            title = f'UMAP Visualization of Weights in Layer {name1} which has {num_filters1} neurons and Square Stable Rank of {square_stable_rank1} and Layer {name2} which has {num_filters2} neurons and Square Stable Rank of {square_stable_rank2}'
            
            plt.figure(figsize=(10, 6))
            plt.scatter(embedding[:, 0], embedding[:, 1], marker='.', c=colors, label=colors)
            plt.title(title)
            plt.xlabel('Component 1')
            plt.ylabel('Component 2')
            plt.show()


In [ ]:
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import time


def display_neurons_plotly_3d(model, reducer):

    # 1. Załaduj przetrenowany model ResNet50
    model.eval()

    # Przejrzyj warstwy konwolucyjne i wydobądź ich wagi
    for (name1, layer1), (name2, layer2) in zip(model.left_branch.named_modules(), model.right_branch.named_modules()):
        if isinstance(layer1, torch.nn.Conv2d) and isinstance(layer2, torch.nn.Conv2d):
            # Pobierz wagi
            dd = input()
            weights1 = layer1.weight.data.cpu().numpy()
            weights2 = layer2.weight.data.cpu().numpy()

            # Spłaszcz wagi
            num_filters1 = weights1.shape[0]
            num_filters2 = weights2.shape[0]
            flattened_weights1 = weights1.reshape(num_filters1, -1)
            flattened_weights2 = weights2.reshape(num_filters2, -1)
            colors = np.where(np.arange(2 * flattened_weights1.shape[0]) < flattened_weights1.shape[0], 'blue', 'red')  # left - blue, right - red
            
            pca1 = PCA()
            singular_values1 = pca1.fit(flattened_weights1).singular_values_
            square_stable_rank1 = round((singular_values1 ** 2).sum() / singular_values1[0] ** 2, 4)
            
            pca2 = PCA()
            singular_values2 = pca2.fit(flattened_weights2).singular_values_
            square_stable_rank2 = round((singular_values2 ** 2).sum() / singular_values2[0] ** 2, 4)

            # 2. Zastosuj UMAP do redukcji wymiarów
            if reducer == 'umap':
                embedding = umap.UMAP(n_components=3, n_neighbors=20, densmap=True).fit_transform(np.concatenate([flattened_weights1, flattened_weights2]))
            elif reducer == 'tsne':
                embedding = TSNE(n_components=3).fit_transform(np.concatenate([flattened_weights1, flattened_weights2]))
            print('DIMS', embedding.shape, flattened_weights1.shape, flattened_weights2.shape)

            title = f'UMAP Visualization of Weights in Layer {name1} which has {num_filters1} neurons and Square Stable Rank of {square_stable_rank1} and Layer {name2} which has {num_filters2} neurons and Square Stable Rank of {square_stable_rank2}'
            fig = go.Figure(data=[go.Scatter3d(x=embedding[:,0], y=embedding[:,1], z=embedding[:,2],
                                       mode='markers',
                                       marker=dict(size=5, opacity=0.8, color=colors))])
            fig.update_layout(title=title)
            clear_output(wait=True)  # Czyszczenie poprzedniego wykresu
            display(fig)            # Wyświetlanie nowego wykresu


In [ ]:
model_name = 'mm_resnet'
checkpoint_path = '/net/pr2/projects/plgrid/plgg_ccbench/bartek/reports2/all_at_once, training with phase1=0, phase2=180, phase3=0, phase4=0, mm_cifar10, mm_resnet, sgd, overlap=0.0_lr=0.5_wd=0.0_lambda=1.0/2023-12-22_03-19-13/checkpoints/model_step_epoch_180.pth'

model_clean = load_model(model_name, checkpoint_path)
display_neurons_2d(model_clean, reducer='tsne')

In [ ]:
model_name = 'mm_resnet'
checkpoint_path = '/net/pr2/projects/plgrid/plgg_ccbench/bartek/reports2/all_at_once, training with phase1=80, phase2=100, phase3=0, phase4=0, mm_cifar10, mm_resnet, sgd, overlap=0.0_lr=0.5_wd=0.0_lambda=1.0/2023-12-22_02-18-18/checkpoints/model_step_epoch_180.pth'

model_dirty = load_model(model_name, checkpoint_path)
display_neurons_via_umap(model_dirty)

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Przykładowe dane
x = np.random.rand(100)
y = np.random.rand(100)
z = np.random.rand(100)

# Tworzenie wykresu
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(size=5,
                                               color=z,  # kolorowanie punktów w zależności od wartości z
                                               colorscale='Viridis',  # wybór palety kolorów
                                               opacity=0.8))])

# Aktualizacja układu wykresu
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), scene=dict(
                    xaxis_title='X Axis',
                    yaxis_title='Y Axis',
                    zaxis_title='Z Axis'))

fig.show()


In [ ]:
model_name = 'mm_resnet'
checkpoint_path = '/net/pr2/projects/plgrid/plgg_ccbench/bartek/reports2/all_at_once, training with phase1=0, phase2=180, phase3=0, phase4=0, mm_cifar10, mm_resnet, sgd, overlap=0.0_lr=0.5_wd=0.0_lambda=1.0/2023-12-22_03-19-13/checkpoints/model_step_epoch_180.pth'

model_dirty = load_model(model_name, checkpoint_path)
display_neurons_plotly_3d(model_dirty, reducer='tsne')

In [ ]:
model_name = 'mm_resnet'
checkpoint_path = '/net/pr2/projects/plgrid/plgg_ccbench/bartek/reports2/all_at_once, training with phase1=80, phase2=100, phase3=0, phase4=0, mm_cifar10, mm_resnet, sgd, overlap=0.0_lr=0.5_wd=0.0_lambda=1.0/2023-12-22_02-18-18/checkpoints/model_step_epoch_180.pth'

model_dirty = load_model(model_name, checkpoint_path)
display_neurons_via_umap_plotly(model_dirty)

In [ ]:
model_name = 'mm_resnet'
checkpoint_path = '/net/pr2/projects/plgrid/plgg_ccbench/bartek/reports2/all_at_once, training with phase1=180, phase2=0, phase3=0, phase4=0, mm_cifar10, mm_resnet, sgd, overlap=0.0_lr=0.5_wd=0.0_lambda=1.0/2023-12-27_06-20-56/checkpoints/model_step_epoch_180.pth'

model_mug = load_model(model_name, checkpoint_path)
display_neurons_via_umap_plotly(model_mug)

In [ ]:
import numpy as np

# Przykładowy zbiór punktów
flattened_weights1 = torch.tensor(np.random.rand(100, 5))  # 1

from sklearn.decomposition import PCA

pca1 = PCA()
singular_values1 = pca1.fit(flattened_weights1).singular_values_
square_stable_rank1 = round((singular_values1 ** 2).sum() / singular_values1[0] ** 2, 4)

# Wyświetlenie wariancji wyjaśnionej przez każdą składową
print("Wariancja wyjaśniona przez każdą składową:", pca1.explained_variance_ratio_, ', SSR:', square_stable_rank1)


In [ ]:
(singular_values1 ** 2).sum() / flattened_weights1.shape[0] - total_variance(flattened_weights1)

In [ ]:
first_conv_layer = model_clean.conv11.weight.data

In [ ]:
first_conv_layer.shape

In [ ]:
import matplotlib.pyplot as plt

# Zakładamy, że wagi są w formacie (liczba_filtrów, kanały, wysokość, szerokość)
num_filters = first_conv_layer.shape[0]

# Ustawienie parametrów wykresu
fig, axes = plt.subplots(8, 8, figsize=(12, 12))
# normalized_weights = (first_conv_layer - first_conv_layer.min()) / (first_conv_layer.max() - first_conv_layer.min())


for i, ax in enumerate(axes.flat):
    # Wyświetlenie wag w siatce 8x8
    if i < first_conv_layer.shape[0]:  # Sprawdzenie, czy i-ty filtr istnieje
        weight_rgb = np.transpose(first_conv_layer[i], (1, 2, 0))
        ax.imshow(weight_rgb)
        ax.axis('off')
    else:
        ax.axis('off')

plt.show()
